# Specify date range and geo-location with .json file,  download all aailable brightness temperature data from Scihub

In [ ]:
# necessary libraries
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date, datetime, timedelta
import time


In [ ]:
# specify username and password, initiate SentinelSat API
your_scihub_username =  '' # fill in coda username here
your_scihub_password = '' # replace with your coda password
api = SentinelAPI(your_scihub_username, your_scihub_password, 'https://scihub.copernicus.eu/dhus')

In [ ]:
# location specified by polygon from .json file
footprint = geojson_to_wkt(read_geojson('Malta_Sicily.json')) # get json file from geojson.io

# specify date range and procut type (level 2 water surface temp.)
# datestrings in format YYYYMMDD
first_date = '20200331'
last_date = '20200608'

# product type
p_type = 'SL_1_RBT___' # product type

# query server to see if files exist
products = api.query(footprint,
                     date=(first_data, last_date),
                     platformname ='Sentinel-3',orbitdirection = 'Descending'\
                     ,producttype =  p_type)
# print number of available products
print(len(products))
# save number of days spanned by first and last dates
N_days = (datetime.strptime(last_date, '%Y%m%d') - datetime.strptime(first_date, '%Y%m%d')).days

In [ ]:
download_path = '' # fill in path to save data to

LTA_dates = []
success_ind = 0 # increment after succesfful download


for ii in range(0,N_days):
    # increment date by one day to provide Sentinelsat with the correct date range
    StartDate = datetime.strptime(first_date,"%Y%m%d") + timedelta(days=ii)
    EndDate = StartDate + timedelta(days=1)
    
    StartDate_str = datetime.strftime(StartDate,"%Y%m%d")
    EndDate_str = datetime.strftime(EndDate,"%Y%m%d")
    
    # query server for data
    products = api.query(footprint,
                     date=(StartDate_str, EndDate_str),
                     platformname ='Sentinel-3',orbitdirection = 'Descending'\
                     ,producttype =  p_type)

    if p_type == 'SL_2_WST___' and len(products)>0:
        
        print('Product online: . Downloading now.')

        products_odata = [api.get_product_odata(product_id) for product_id in products]
        for ii in range(0,len(products)):
            api.download(products_odata[ii]['id'],directory_path = download_path);

            success_ind += 1
        
    
    # check to see if product exist for time and location specified
    
    
    elif  p_type == 'SL_1_RBT___' and len(products)>0:
        
            list_of_items = list(products.items()) # items from ordered dictionary
            product_ID = list_of_items[0][0] # item corresponding to product id
            product_info = api.get_product_odata(product_ID) # check to see if product id is online



            if product_info['Online']: # binary check for online status
                print('Product online: . Downloading now.')

                # if product(s) exist download it/them
                api.download_all(products,directory_path=download_path)
                
                success_ind += 1

            # if not online, save the date for later downloading and wait to make next query
            # possibly no necessary for CODA
            else:
                # try to download to trigger data recall from LTA
                api.download_all(products,directory_path=download_path)
                print('Product offline: Waiting 30 mins for next request.')
                LTA_dates.append(StartDate_str)
                time.sleep(30.5*60) # wait 30.5 mins to make next server query